In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dropout, Layer
from tensorflow.keras.layers import Embedding, Input, GlobalAveragePooling1D, Dense
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential, Model
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

In [13]:
class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Sequential(
            [Dense(ff_dim, activation="relu"), 
             Dense(embed_dim),]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [14]:
class TokenAndPositionEmbedding(Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [15]:
vocab_size = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each movie review

(x_train, y_train), (x_val, y_val) = imdb.load_data(num_words=vocab_size)

print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")

25000 Training sequences
25000 Validation sequences


In [16]:
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_val = tf.keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)

In [20]:
# read data.csv
import pandas as pd
from sklearn.model_selection import train_test_split

# df = pd.read_csv('data.csv')
df = pd.read_csv('spam_email.csv')

print(df.head())

                                                Text  CB_Label
0  Subject: stock promo mover : cwtd\n * * * urge...         1
1  Subject: are you listed in major search engine...         1
2  Subject: important information thu , 30 jun 20...         1
3  Subject: = ? utf - 8 ? q ? bask your life with...         1
4  Subject: " bidstogo " is places to go , things...         1


In [21]:
# Tokenize the sentences
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(df['Text'])

# Obtain the corresponding word indices
x_train = tokenizer.texts_to_sequences(df['Text'])
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)

# Obtain the corresponding labels
y_train = df['CB_Label'].values

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)

print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")


8000 Training sequences
2000 Validation sequences


In [22]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = GlobalAveragePooling1D()(x)
x = Dropout(0.1)(x)
x = Dense(20, activation="relu")(x)
x = Dropout(0.1)(x)
outputs = Dense(2, activation="softmax")(x)

model = Model(inputs=inputs, outputs=outputs)

In [23]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

history = model.fit(x_train, y_train, 
                    batch_size=64, epochs=3, 
                    validation_data=(x_val, y_val)
                   )

Epoch 1/3
125/125 [==============================] - 15s 109ms/step - loss: 0.3202 - accuracy: 0.8418 - val_loss: 0.0975 - val_accuracy: 0.9765
Epoch 2/3
125/125 [==============================] - 14s 111ms/step - loss: 0.0583 - accuracy: 0.9805 - val_loss: 0.0793 - val_accuracy: 0.9690
Epoch 3/3
125/125 [==============================] - 15s 116ms/step - loss: 0.0302 - accuracy: 0.9899 - val_loss: 0.0859 - val_accuracy: 0.9690


In [ ]:
model.save_weights("predict_class.h5")

In [24]:
results = model.evaluate(x_val, y_val, verbose=2)

for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

63/63 - 1s - loss: 0.0859 - accuracy: 0.9690 - 1s/epoch - 21ms/step
loss: 0.086
accuracy: 0.969
